<a href="https://colab.research.google.com/github/devroopsaha744/HateSpeechDetect-text/blob/main/Hate_Speech_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate -U -q
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, recall_score, f1_score, precision_score
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [3]:
!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [5]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels = 3)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
data = load_dataset("tdavidson/hate_speech_offensive")
data = data['train']
data_df = data.to_pandas()

Generating train split:   0%|          | 0/24783 [00:00<?, ? examples/s]

In [7]:
# Load the stop words
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words and lemmatize the tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Join the tokens back into a single string
    text = ' '.join(tokens)
    return text

# Apply the preprocessing function to the 'text' column
data_df['tweet'] = data_df['tweet'].apply(preprocess_text)

In [8]:
data_df

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,3,0,0,3,2,rt mayasolovely woman shouldnt complain cleani...
1,3,0,3,0,1,rt mleew17 boy dat coldtyga dwn bad cuffin dat...
2,3,0,3,0,1,rt urkindofbrand dawg rt 80sbaby4life ever fuc...
3,3,0,2,1,1,rt cganderson vivabased look like tranny
4,6,0,6,0,1,rt shenikaroberts shit hear might true might f...
...,...,...,...,...,...,...
24778,3,0,2,1,1,yous muthafin lie 8220lifeasking 20pearls core...
24779,3,0,1,2,2,youve gone broke wrong heart baby drove rednec...
24780,3,0,3,0,1,young buck wan na eat dat nigguh like aint fuc...
24781,6,0,6,0,1,youu got wild bitch tellin lie


In [9]:
x = list(data_df['tweet'])
y = list(data_df['class'])

In [10]:
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size = 0.3)

In [11]:
x_train_tokenized = tokenizer(x_train, padding = True, truncation = True, max_length = 512 )

In [12]:
x_val_tokenized = tokenizer(x_val, padding = True, truncation = True, max_length = 512 )

In [13]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [14]:
train_dataset = Dataset(x_train_tokenized, y_train)
val_dataset = Dataset(x_val_tokenized, y_val)

In [15]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    clf_rep = classification_report(y_true=labels, y_pred=pred)

    return {"classification_report": clf_rep}

In [16]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=2,
    per_device_train_batch_size=32

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [17]:
trainer.train()

Step,Training Loss
500,0.312700
1000,0.223200


TrainOutput(global_step=1086, training_loss=0.2644719357429069, metrics={'train_runtime': 898.3854, 'train_samples_per_second': 38.62, 'train_steps_per_second': 1.209, 'total_flos': 2870637276914064.0, 'train_loss': 0.2644719357429069, 'epoch': 2.0})

In [18]:
trainer.evaluate()

Trainer is attempting to log a value of "              precision    recall  f1-score   support

           0       0.47      0.20      0.29       420
           1       0.93      0.97      0.95      5732
           2       0.88      0.90      0.89      1283

    accuracy                           0.91      7435
   macro avg       0.76      0.69      0.71      7435
weighted avg       0.90      0.91      0.90      7435
" of type <class 'str'> for key "eval/classification_report" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.24802963435649872,
 'eval_classification_report': '              precision    recall  f1-score   support\n\n           0       0.47      0.20      0.29       420\n           1       0.93      0.97      0.95      5732\n           2       0.88      0.90      0.89      1283\n\n    accuracy                           0.91      7435\n   macro avg       0.76      0.69      0.71      7435\nweighted avg       0.90      0.91      0.90      7435\n',
 'eval_runtime': 37.7818,
 'eval_samples_per_second': 196.788,
 'eval_steps_per_second': 24.615,
 'epoch': 2.0}

### **Metrics for Bert**
Trainer is attempting to log a value of "             
                precision   recall  f1-score   support

           0       0.47      0.20      0.29       420
           1       0.93      0.97      0.95      5732
           2       0.88      0.90      0.89      1283

    accuracy                           0.91      7435
   macro avg       0.76      0.69      0.71      7435
weighted avg       0.90      0.91      0.90      7435